In [10]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Utilities 
from options.options import is_notebook, get_options, update_options, print_options
import argparse
import math
import time
import random
import os
from tqdm import tqdm

# Data
import numpy as np
from sklearn.model_selection import train_test_split
from datautils.ts_dataset import TSDataset
from datautils.data import prepare_dataloaders
import pandas as pd

# Torch
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Transformer
import transformer.Constants as Constants
from transformer.Models import Transformer
from transformer.Optim import ScheduledOptim

%autoreload 2

In [12]:
opt = get_options()
print_options(opt)

           train_path :  /home/yannic/master-thesis/data_air/prsa_data.parquet
             val_path :  /home/yannic/master-thesis/data_air/prsa_data.parquet
                epoch :  10
           batch_size :  32
              d_model :  512
       d_inner_hidden :  2048
                d_key :  64
              d_value :  64
           d_sequence :  512
               n_head :  8
             n_layers :  6
       n_warmup_steps :  4000
               lr_mul :  2.0
                 seed :  False
              dropout :  0.1
    embs_share_weight :  True
    proj_share_weight :  True
     scale_emb_or_prj :  prj
           output_dir :  ./output
               use_tb :  True
            save_mode :  best
                 cuda :  True
      label_smoothing :  True


In [13]:
# https://pytorch.org/docs/stable/notes/randomness.html
# For reproducibility
if opt["seed"] is not None:
    torch.manual_seed(opt["seed"])
    torch.backends.cudnn.benchmark = False
    # torch.set_deterministic(True)
    np.random.seed(opt["seed"])
    random.seed(opt["seed"])

In [14]:
if not opt["output_dir"]:
    print('No experiment result will be saved.')
    raise

In [15]:
if not os.path.exists(opt["output_dir"]):
    os.makedirs(opt["output_dir"])

In [16]:
# define the cuda devide
device = torch.device('cuda' if opt["cuda"] else 'cpu')

In [17]:
# Load the dataset and preprocess it 
aq = pd.read_parquet("/home/yannic/master-thesis/data_air/prsa_data.parquet")
aq_prep = pd.concat([aq,pd.get_dummies(aq['station'], prefix='station',dummy_na=False)],axis=1).drop(['station'],axis=1).drop(columns=["wind_direction"])

# Functions

In [ ]:
def train(model, training_data, validation_data, optimizer, device, opt):
    ''' Start training '''

    # Use tensorboard to plot curves, e.g. perplexity, accuracy, learning rate
    if opt.use_tb:
        print("[Info] Use Tensorboard")
        from torch.utils.tensorboard import SummaryWriter
        tb_writer = SummaryWriter(log_dir=os.path.join(opt.output_dir, 'tensorboard'))

    log_train_file = os.path.join(opt.output_dir, 'train.log')
    log_valid_file = os.path.join(opt.output_dir, 'valid.log')

    print('[Info] Training performance will be written to file: {} and {}'.format(
        log_train_file, log_valid_file))

    with open(log_train_file, 'w') as log_tf, open(log_valid_file, 'w') as log_vf:
        log_tf.write('epoch,loss,ppl,accuracy\n')
        log_vf.write('epoch,loss,ppl,accuracy\n')

    def print_performances(header, ppl, accu, start_time, lr):
        print('  - {header:12} ppl: {ppl: 8.5f}, accuracy: {accu:3.3f} %, lr: {lr:8.5f}, '\
              'elapse: {elapse:3.3f} min'.format(
                  header=f"({header})", ppl=ppl,
                  accu=100*accu, elapse=(time.time()-start_time)/60, lr=lr))

    #valid_accus = []
    valid_losses = []
    for epoch_i in range(opt.epoch):
        print('[ Epoch', epoch_i, ']')

        start = time.time()
        train_loss, train_accu = train_epoch(
            model, training_data, optimizer, opt, device, smoothing=opt.label_smoothing)
        train_ppl = math.exp(min(train_loss, 100))
        # Current learning rate
        lr = optimizer._optimizer.param_groups[0]['lr']
        print_performances('Training', train_ppl, train_accu, start, lr)

        start = time.time()
        valid_loss, valid_accu = eval_epoch(model, validation_data, device, opt)
        valid_ppl = math.exp(min(valid_loss, 100))
        print_performances('Validation', valid_ppl, valid_accu, start, lr)

        valid_losses += [valid_loss]

        checkpoint = {'epoch': epoch_i, 'settings': opt, 'model': model.state_dict()}

        if opt.save_mode == 'all':
            model_name = 'model_accu_{accu:3.3f}.chkpt'.format(accu=100*valid_accu)
            torch.save(checkpoint, model_name)
        elif opt.save_mode == 'best':
            model_name = 'model.chkpt'
            if valid_loss <= min(valid_losses):
                torch.save(checkpoint, os.path.join(opt.output_dir, model_name))
                print('    - [Info] The checkpoint file has been updated.')

        with open(log_train_file, 'a') as log_tf, open(log_valid_file, 'a') as log_vf:
            log_tf.write('{epoch},{loss: 8.5f},{ppl: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=train_loss,
                ppl=train_ppl, accu=100*train_accu))
            log_vf.write('{epoch},{loss: 8.5f},{ppl: 8.5f},{accu:3.3f}\n'.format(
                epoch=epoch_i, loss=valid_loss,
                ppl=valid_ppl, accu=100*valid_accu))

        if opt.use_tb:
            tb_writer.add_scalars('ppl', {'train': train_ppl, 'val': valid_ppl}, epoch_i)
            tb_writer.add_scalars('accuracy', {'train': train_accu*100, 'val': valid_accu*100}, epoch_i)
            tb_writer.add_scalar('learning_rate', lr, epoch_i)


# DataLoader and Transformer

In [25]:
# Load Dataset
train, test = train_test_split(aq_prep, test_size=0.10)

train_dataset, train_dataloader = prepare_dataloaders(train.values, opt["batch_size"])
test_dataset, test_dataloader = prepare_dataloaders(test.values,  opt["batch_size"])

opt["src_sequence_size"] = 200
opt["trg_sequence_size"] = 200
opt["src_pad_idx"] = 0
opt["trg_pad_idx"] = 0

# Define Transformer
transformer = Transformer(
        opt["src_sequence_size"],
        opt["trg_sequence_size"],
        src_pad_idx=opt["src_pad_idx"],
        trg_pad_idx=opt["trg_pad_idx"],
        trg_emb_prj_weight_sharing=opt["proj_share_weight"],
        emb_src_trg_weight_sharing=opt["embs_share_weight"],
        d_k=opt["d_key"],
        d_v=opt["d_value"],
        d_model=opt["d_model"],
        d_word_vec=opt["d_sequence"],
        d_inner=opt["d_inner_hidden"],
        n_layers=opt["n_layers"],
        n_head=opt["n_head"],
        dropout=opt["dropout"],
        scale_emb_or_prj=opt["scale_emb_or_prj"]).to(device)

# Define Optimizer
optimizer = ScheduledOptim(
                optim.Adam(transformer.parameters(), betas=(0.9, 0.98), eps=1e-09),
                opt["lr_mul"], opt["d_model"], opt["n_warmup_steps"])

# Start the training process
#train(transformer, training_data, validation_data, optimizer, device, opt)